# Imports


In [1]:
import requests
import time
import json
import math
import os
import threading

# Specify Contest

In [2]:
CONTEST = 'weekly-contest-242'
CONTEST_NAME = 'Weekly Contest 244'
PAGE_LIMIT = None
os.makedirs(f'{CONTEST_NAME}',exist_ok=True)
os.makedirs(f'{CONTEST_NAME}/Cache/',exist_ok=True)

# Headers for fetching data

In [3]:
headers = {
    'authority': 'leetcode.com',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-newrelic-id': 'UAQDVFVRGwEAXVlbBAg=',
    'x-requested-with': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    'content-type': 'application/json',
    'sec-gpc': '1',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'csrftoken=SYt4auAHmprCYhMeGDZwbnE0hWoSLKRLHb0A7mMpbvoAEPvjYcIdL5LtaWhQJv6B; __cfduid=dc8614cf2c524abada1e54cfa8e118d271619065816; __cf_bm=08dec7596eec629845d4f3f39086483d491e2c40-1619324757-1800-AYKgjE4h5oPn/k7bslI317JZOCKGqrKvRQEJSKFM8EyY57yFeDgrM0KTqzs/c5RUfzN8A7VceydacsvxJjrmrp4=',
}

In [6]:
PAGE_LIMIT = getPageNo(CONTEST)
fromCN = set()
print(PAGE_LIMIT)

496


# Helper Functions to get Ranklist

In [5]:
def getPageNo(contest_slug):
    params = (
        ('pagination', '1'),
        ('region', 'global'),
    )
    data = json.loads(requests.get('https://leetcode.com/contest/api/ranking/{}/'.format(contest_slug), headers=headers, params=params).content)
    return int(math.ceil(data['user_num']/25))

In [7]:
def getPage(page,data2,visit):
    if page > PAGE_LIMIT:
        return
    if page in visit:
        return
    params = (
        ('pagination', str(page)),
        ('region', 'global'),
    )
    try:
        data = json.loads(requests.get('https://leetcode.com/contest/api/ranking/{}/'.format(CONTEST), headers=headers, params=params).content)
    except:
        print("error at page",page)
        return 
    res = []
    for x in data['total_rank']:
        if int(x['score']) ==0:
            continue
        if x['data_region'] == 'CN':
            fromCN.add(x['username'])
        res.append([x['rank'],x['username']])
    data2.extend(res)
    visit.add(page)
    return

In [8]:
def getRanklist(contest):
    res = []
    if os.path.isfile(f'{contest}_all.json'):
        res = json.loads(open(f'{contest}_all.json').read())
        print("Found Old Ranklist... Using That")
        return res
    print("Ranklist not found... Trying to fetch new")
    visit=set()
    for i in range(1,1000,50):
        THREADS = []
        for j in range(i,i+50):
            THREADS.append(threading.Thread(target=getPage, args=(j,res,visit,)))
        for j in range(len(THREADS)):
            THREADS[j].start()
        for j in range(len(THREADS)):
            THREADS[j].join()
        print(f'{i}')
    res = sorted(res, key=lambda x: x[0])
    with open(f'{contest}_all.json','w') as f:
        f.write(json.dumps(res))
    return res

In [9]:
ranklist = getRanklist(CONTEST)

Ranklist not found... Trying to fetch new
1
51
101
151
201
251
301
351
401
451
501
551
601
651
701
751
801
851
901
951


# Helper Functions to get User Data

In [10]:
def getUserData_US(username,get_current=False):
    data = '{"operationName":"getContestRankingData","variables":{"username":\"'+username+'\"},"query":"query getContestRankingData($username: String!) {\\n  userContestRanking(username: $username) {\\n    attendedContestsCount\\n    rating\\n    globalRanking\\n    __typename\\n  }\\n  userContestRankingHistory(username: $username) {\\n    contest {\\n      title\\n      startTime\\n      __typename\\n    }\\n    rating\\n    ranking\\n    __typename\\n  }\\n}\\n"}'
    response = requests.post('https://leetcode.com/graphql', headers=headers, data=data).content
    data = json.loads(response)
    allContests = data['data']['userContestRankingHistory']
    user_k=0
    user_rating=None
    for x in allContests:
        name = x['contest']['title']
        ranking = x['ranking']
        rating = x['rating']
        if get_current==False and name == CONTEST_NAME:
            break
        user_rating = rating
        if ranking!=0:
            user_k+=1
    return (user_rating,user_k)

In [11]:
def getUserData_CN(username,get_current=False):
    data = '{"operationName":"userContest","variables":{"userSlug":\"'+username+'\"},"query":"query userContest($userSlug: String!) {\\n  userContestRanking(userSlug: $userSlug) {\\n    currentRatingRanking\\n    ratingHistory\\n    levelHistory\\n    contestRankingHistoryV2\\n contestHistory\\n    __typename\\n  }\\n  globalRatingRanking(userSlug: $userSlug)\\n  userContestScore(userSlug: $userSlug)\\n  contestUnratedContests\\n}\\n"}'
    response = requests.post('https://leetcode-cn.com/graphql', headers=headers, data=data).content
    data = json.loads(response)
    allContests = [x['title_slug'] for x in json.loads(data['data']['userContestRanking']['contestHistory'])]
    allRating = json.loads(data['data']['userContestRanking']['ratingHistory'])
    user_k=0
    user_rating=None
    for i in range(len(allContests)):
        name = allContests[i]
        rating = allRating[i]
        if get_current==False and  name == CONTEST:
            break
        user_rating = rating
        if rating!=None:
            user_k+=1
    return (user_rating,user_k)

In [12]:
def getUserData(username):
    if os.path.isfile(f'{CONTEST_NAME}/Cache/{username}.json') == True:
        cur = json.loads(open(f'{CONTEST_NAME}/Cache/{username}.json').read())
        return cur
    if username in fromCN:
        try:
            cur  = getUserData_CN(username)
            with open(f'{CONTEST_NAME}/Cache/{username}.json','w') as f:
                f.write(json.dumps(cur))
        except:
            pass
        return
    try:
        cur  = getUserData_US(username)
        with open(f'{CONTEST_NAME}/Cache/{username}.json','w') as f:
            f.write(json.dumps(cur))
    except:
        try:
            cur  = getUserData_CN(username)
            with open(f'{CONTEST_NAME}/Cache/{username}.json','w') as f:
                f.write(json.dumps(cur))
        except:
            pass

In [13]:
def fetchAllUserData(ranklist):
    lssize = len(ranklist)
    userData = {}
    for i in range(0,lssize,20):
        THREADS = []
        for j in range(i,i+20):
            if j>=lssize:
                break
            THREADS.append(threading.Thread(target=getUserData, args=(ranklist[j][1],)))
        for j in range(len(THREADS)):
            THREADS[j].start()
        for j in range(len(THREADS)):
            THREADS[j].join()
        print(f"Done : {i}/{lssize}")
    for i in range(0,lssize):
        username = ranklist[i][1]
        if os.path.isfile(f'{CONTEST_NAME}/Cache/{username}.json') == True:
            cur = json.loads(open(f'{CONTEST_NAME}/Cache/{username}.json').read())
            userData[username]=cur
    return userData

In [ ]:
userData = fetchAllUserData(ranklist)

Done : 0/8632


# Helper Functions to do rating calculation

In [21]:
def getRatingChange(username):
    def E(a,b):
        return 1/(1+math.pow(10,(b-a)/400))
    
    def getActualRank(username):
        for i in range(len(ranklist)):
            if ranklist[i][1]==username:
                return i+1
        return None

    def getERank(username):
        ans = 1
        for i in range(len(ranklist)):
            if ranklist[i][1]!=username:
                try:
                    ans+= ( (E(userData[ranklist[i][1]][0],userData[username][0])) )
                except:
                    pass
        return ans
    
    def getMi(Erank,Rank):
        return math.sqrt(Rank*Erank)
    
    def checkBS(Erating):
        ans = 1
        for i in range(len(ranklist)):
            try:
                ans+= ( (E(userData[ranklist[i][1]][0],Erating)) )
            except:
                pass
        return ans
    
    def getErating(rating,m):
        l=1
        r=100000
        while r-l > 0.1:
            mid = (l+r)/2
            cur = checkBS(mid)
            if cur>m:
                l=mid
            else:
                r=mid
        return l
    
    def f(k):
        if k==1:
            return 0.5
        return 2/9
    
    def getDelta(Erating,rating,k):
        try:
            return f(k)*(Erating-rating)
        except:
            return None
    try:
        ERank = getERank(username)
        ar = getActualRank(username)
        mi = getMi(ERank,ar)
        ERating = getErating(userData[username][0],mi)
        delta = getDelta(ERating,userData[username][0],userData[username][1])
        if delta==None:
            return [None,None]
        newrating = userData[username][0] + delta
        ratingChanges.append({'rank':ar,'username':username,'newrating':newrating,'delta':delta})
    except:
        pass
    return

In [16]:
ratingChanges = []
def getRatingChanges():
    if os.path.isfile(f'{CONTEST}_rating_changes.json'):
        return json.loads(open(f'{CONTEST}_rating_changes.json').read())
    ranklist_size = len(ranklist)
    for i in range(0,ranklist_size,50):
        THREADS = []
        for j in range(i,i+50):
            if j>=ranklist_size:
                break
            THREADS.append(threading.Thread(target=getRatingChange, args=(ranklist[j][1],)))        
        for j in range(len(THREADS)):
            THREADS[j].start()
        for j in range(len(THREADS)):
            THREADS[j].join()
        print(f"{i+50}/{ranklist_size}")
    
    return ratingChanges

In [ ]:
getRatingChanges()

# Save Rating Changes

In [17]:
ratingChanges = sorted(ratingChanges, key=lambda x: x['rank'])
with open(f'{CONTEST}_rating_changes.json','w') as f:
    f.write(json.dumps(ratingChanges))